In [1]:
import os

old_folder = r'D:\Cosmos_MOp\20240918_Mecp2_27wk_visual\Alignment'

before_position_file = os.path.join(old_folder, '10x_positions_before.txt')

new_folder = r'D:\Cosmos_MOp\20241026_DNA_from_Mecp2_0918\Alignment_H1C2'

after_position_file = os.path.join(new_folder, '10x_positions_after.txt')

os.path.isfile(before_position_file), os.path.isfile(after_position_file)



(True, True)

# Calculate rotation matrix

In [2]:
import numpy as np
import os
# 1. alignment for manually picked points
def align_manual_points(pos_file_before, pos_file_after,
                        save=True, save_folder=None, save_filename='', verbose=True):
    """Function to align two manually picked position files, 
    they should follow exactly the same order and of same length.
    Inputs:
        pos_file_before: full filename for positions file before translation
        pos_file_after: full filename for positions file after translation
        save: whether save rotation and translation info, bool (default: True)
        save_folder: where to save rotation and translation info, None or string (default: same folder as pos_file_before)
        save_filename: filename specified to save rotation and translation points
        verbose: say something! bool (default: True)
    Outputs:
        R: rotation for positions, 2x2 array
        T: traslation of positions, array of 2
    Here's example for how to translate points
        translated_ps_before = np.dot(ps_before, R) + t
    """
    # load position_before
    if os.path.isfile(pos_file_before):
        ps_before = np.loadtxt(pos_file_before, delimiter=',')

    # load position_after
    if os.path.isfile(pos_file_after):
        ps_after = np.loadtxt(pos_file_after, delimiter=',')

    # do SVD decomposition to get best fit for rigid-translation
    c_before = np.mean(ps_before, axis=0)
    c_after = np.mean(ps_after, axis=0)
    H = np.dot((ps_before - c_before).T, (ps_after - c_after))
    U, _, V = np.linalg.svd(H)  # do SVD
    # calcluate rotation
    R = np.dot(V, U.T).T
    if np.linalg.det(R) < 0:
        R[:, -1] = -1 * R[:, -1]
    # calculate translation
    t = - np.dot(c_before, R) + c_after
    # here's example for how to translate points
    # translated_ps_before = np.dot(ps_before, R) + t
    # save
    if save:
        if save_folder is None:
            save_folder = os.path.dirname(pos_file_before)
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)
        if len(save_filename) > 0:
            save_filename += '_'
        rotation_name = os.path.join(save_folder, save_filename+'rotation')
        translation_name = os.path.join(
            save_folder, save_filename+'translation')
        np.save(rotation_name, R)
        np.save(translation_name, t)

    return R, t

In [3]:
R, T = align_manual_points(before_position_file, after_position_file, save=False, save_folder=new_folder)
R, T

(array([[ 0.99991867,  0.01275391],
        [-0.01275391,  0.99991867]]),
 array([-1519.60505975,  1284.62351792]))

In [4]:
np.arccos(R[0,1])

1.5580420755147795

# Translate 60x positions

In [5]:
old_positions = np.loadtxt(os.path.join(os.path.dirname(old_folder), 'positions_all.txt'), delimiter=',')

In [6]:
new_positions = np.dot(old_positions, R) + T
print(new_positions)

[[-4497.71094971   160.54961103]
 [-4500.32550034   365.53293748]
 [-4502.94005097   570.51626394]
 [-4505.55460159   775.49959039]
 [-4508.16915222   980.48291685]
 [-4510.78370284  1185.46624331]
 [-4513.39825347  1390.44956976]
 [-4308.41492701  1393.06412039]
 [-4305.80037639  1188.08079393]
 [-4303.18582576   983.09746748]
 [-4300.57127514   778.11414102]
 [-4297.95672451   573.13081456]
 [-4295.34217388   368.14748811]
 [-4292.72762326   163.16416165]
 [-4085.12974618   -39.20461418]
 [-4087.7442968    165.77871228]
 [-4090.35884743   370.76203873]
 [-4092.97339805   575.74536519]
 [-4095.58794868   780.72869165]
 [-4098.2024993    985.7120181 ]
 [-4100.81704993  1190.69534456]
 [-3895.83372347  1193.30989518]
 [-3893.21917285   988.32656873]
 [-3890.60462222   783.34324227]
 [-3887.9900716    578.35991582]
 [-3885.37552097   373.37658936]
 [-3882.76097035   168.3932629 ]
 [-3880.14641972   -36.59006355]
 [-3672.54854264  -238.95883938]
 [-3675.16309326   -33.97551293]
 [-3677.77

In [7]:
save_filename = os.path.join(new_folder, 'translated_positions_all.txt')
print(save_filename)
np.savetxt(save_filename, new_positions, fmt='%.2f', delimiter=',')

D:\Cosmos_MOp\20241026_DNA_from_Mecp2_0918\Alignment_H1C2\translated_positions_all.txt


# Manual adjust positions

In [8]:
manual_positions_before_file = os.path.join(os.path.dirname(old_folder), '60x_positions_before.txt')
print(manual_positions_before_file)
manual_positions = np.loadtxt(manual_positions_before_file, delimiter=',')
print(manual_positions)

D:\Cosmos_MOp\20240918_Mecp2_27wk_visual\60x_positions_before.txt
[[-4529.14 -1167.52]
 [-2580.61  2394.28]
 [ 2981.92  2598.88]
 [ 2871.93 -1764.13]]


In [9]:
translated_manual_positions = np.dot(manual_positions, R) + T
print('translated manual positions:')
print(np.round(translated_manual_positions, 1))

translated manual positions:
[[-6033.5    59.4]
 [-4130.5  3645.8]
 [ 1428.9  3921.3]
 [ 1374.6  -442.7]]


In [10]:
manual_real_positions = [
    [-6078, 91],
    [-4165, 3678],
    [1390, 3932],
    [1333, -436],
]
manual_shifts = np.array(manual_real_positions) - translated_manual_positions[:len(manual_real_positions)]
manual_shifts

array([[-44.51375469,  31.5657461 ],
       [-34.45841167,  32.20407536],
       [-38.92653779,  10.67673445],
       [-41.59090096,   6.73467389]])

In [11]:
manual_shift = np.mean(manual_shifts, axis=0)
manual_shift

array([-39.87240128,  20.29530745])

In [12]:
adjusted_new_positions = new_positions + manual_shift

In [13]:
adjusted_new_positions[0], old_positions[0]

(array([-4537.58335099,   180.84491848]), array([-2992.2, -1086. ]))

In [14]:
adj_save_filename = os.path.join(new_folder, 'adjusted_translated_positions_all.txt')
print(adj_save_filename)
np.savetxt(adj_save_filename, adjusted_new_positions, fmt='%.2f', delimiter=',')

D:\Cosmos_MOp\20241026_DNA_from_Mecp2_0918\Alignment_H1C2\adjusted_translated_positions_all.txt
